# Setup

In [ ]:
!pip install jupyter-dash

     |████████████████████████████████| 8.5 MB 7.0 MB/s 
     |████████████████████████████████| 357 kB 44.7 MB/s 
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=121e1a01b8c83a5787e2b8c1e928ac0eb50638b9e5807583907f647f143560c2
  Stored in directory: /root/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying


In [ ]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objects as go

import json
import pandas as po
import numpy as np
data = po.read_csv("data/Cleaned_airports.csv", index_col=0, parse_dates=True)
data['Airport.Name'] = data['Airport.Name'].str.strip()
data.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  after removing the cwd from sys.path.


,Airport.Code,Airport.Location,Airport.Name,Statistics.# of Delays.Carrier,Statistics.# of Delays.Late Aircraft,Statistics.# of Delays.National Aviation System,Statistics.# of Delays.Security,Statistics.# of Delays.Total,Statistics.# of Delays.Weather,Statistics.Carriers.Names,...,Statistics.Minutes Delayed.Carrier,Statistics.Minutes Delayed.Late Aircraft,Statistics.Minutes Delayed.National Aviation System,Statistics.Minutes Delayed.Security,Statistics.Minutes Delayed.Total,Statistics.Minutes Delayed.Weather,Time.Label,Time.Month,Time.Month Name,Time.Year
0,ATL,"Atlanta, GA",Hartsfield-Jackson Atlanta International,1009,1275,3217,17,5846,328,"American Airlines Inc.,JetBlue Airways,Contine...",...,61606,68335,118831,518,268764,19474,2003-06-01,6,June,2003
1,TPA,"Tampa, FL",Tampa International,226,327,524,3,1122,42,"American Airlines Inc.,JetBlue Airways,Contine...",...,11012,18558,19078,112,51364,2604,2003-06-01,6,June,2003
2,SLC,"Salt Lake City, UT",Salt Lake City International,363,290,352,15,1056,36,"American Airlines Inc.,JetBlue Airways,Contine...",...,16227,13883,10512,556,42863,1685,2003-06-01,6,June,2003
3,SFO,"San Francisco, CA",San Francisco International,416,312,1080,14,1881,59,"American Airlines Inc.,Alaska Airlines Inc.,Co...",...,22673,17879,45407,1138,90487,3390,2003-06-01,6,June,2003
4,SEA,"Seattle, WA",Seattle/Tacoma International,448,552,454,28,1518,36,"American Airlines Inc.,Alaska Airlines Inc.,Je...",...,22647,28119,13860,1097,68673,2950,2003-06-01,6,June,2003


In [ ]:
df = data.copy()
df['Statistics.Flights.Proportion Delayed'] = df['Statistics.Flights.Delayed'] / df['Statistics.Flights.Total']

chances_delay = None
with open("outputs/airport-delay_chance.json", "r") as f:
    chances_delay = po.Series(json.load(f))

In [ ]:
geo_df = po.read_csv("outputs/geopandas_latlon.csv",index_col=0)
geo_df.head()

,Airport.Name,Latitude,Longitude,Chance of Delay,geometry
0,Salt Lake City International,40.7899,-111.9791,15.030975,POINT (-111.9791 40.7899)
1,Phoenix Sky Harbor International,33.4352,-112.0101,16.309776,POINT (-112.0101 33.4352)
2,Chicago Midway International,41.7868,-87.7522,17.338881,POINT (-87.7522 41.7868)
3,Denver International,39.8561,-104.6737,17.596135,POINT (-104.6737 39.8561)
4,Dallas/Fort Worth International,32.8998,-97.0403,17.706570,POINT (-97.0403 32.8998)


# Build Dashboard
Displays proportion of delayed flights over time for each airport.

Additionally displays least and most likely airports to have a delayed flight.

In [ ]:
# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.Div([
        html.H1("Airport Dashboard"),
        html.Label([
            "Airport Name",
            dcc.Dropdown(
                id='airport-name-dropdown', clearable=False,
                value='San Francisco International', options=[
                    {'label': name, 'value': name}
                    for name in df['Airport.Name'].sort_values().unique()
                ])
            ]),
        dcc.Graph(id='graph'),
    ]),
    html.Div([
        html.H2("Reasons for Delay Breakdown"),
        dcc.Graph(id='donut-graph')
    ]),
    html.Div([
      html.Div([
          html.H2("Airports least likely to have a flight delay"),
          html.H3("{}: {}%".format(chances_delay.index[0], round(100*chances_delay.iloc[0], 2))),
          html.H3("{}: {}%".format(chances_delay.index[1], round(100*chances_delay.iloc[1],2))),
          html.H3("{}: {}%".format(chances_delay.index[2], round(100*chances_delay.iloc[2],2)))
      ],style={'width': '49%', 'display': 'inline-block'}),
      html.Div([
          html.H2("Airports most likely to have a flight delay"),
          html.H3("{}: {}%".format(chances_delay.index[-1], round(100*chances_delay.iloc[-1], 2))),
          html.H3("{}: {}%".format(chances_delay.index[-2], round(100*chances_delay.iloc[-2],2))),
          html.H3("{}: {}%".format(chances_delay.index[-3], round(100*chances_delay.iloc[-3],2)))
      ],style={'width': '49%', 'display': 'inline-block'}),
    ])

])
# Define callbacks to update graphs
@app.callback(
    Output('graph', 'figure'),
    [Input("airport-name-dropdown", "value")]
)
def update_figure(val):
    try:
        #print(val)
        d = df[df['Airport.Name'] == val]
        return px.line(
            d, x="Time.Label", y="Statistics.Flights.Proportion Delayed",
            render_mode="webgl", title="Proportion of Delayed Flights over Time"
        )
    except:
        return None


@app.callback(
    Output('donut-graph', 'figure'),
    [Input("airport-name-dropdown", "value")]
)
def update_donut(val):
    try:
        d = df[df['Airport.Name'] == val]
        total = d['Statistics.# of Delays.Total'].sum() + 1e-6 # avoid divide by 0 error during initialization

        labels = ['Carrier', 'Late Aircraft', 'National Aviation System', 'Security', 'Weather']
        values = [d['Statistics.# of Delays.{}'.format(col)].sum() / total for col in labels]
        return go.Figure(
            data=[go.Pie(labels=labels, values=values, hole=.35,)]
        )
    except:
        return None


In [ ]:
# Run app and display result inline in the notebook
app.run_server(mode='inline')

<IPython.core.display.Javascript object>

# Attempt to Build Another Dashboard
Displays interactive US map from the Web Scraping notebook, allows for seeing how likely airports have delays due to [weather, late carrier, national aviation system, etc.]; additionally, can provide visualizations for these statistics in different seasons

In [ ]:
# of delays: what's the chance you will have to wait
# minutes delayed: how long will you have to wait
# looking at months/seasons: when's the best time to schedule flights

df.columns

Index(['Airport.Code', 'Airport.Location', 'Airport.Name',
       'Statistics.# of Delays.Carrier',
       'Statistics.# of Delays.Late Aircraft',
       'Statistics.# of Delays.National Aviation System',
       'Statistics.# of Delays.Security', 'Statistics.# of Delays.Total',
       'Statistics.# of Delays.Weather', 'Statistics.Carriers.Names',
       'Statistics.Carriers.Total', 'Statistics.Flights.Cancelled',
       'Statistics.Flights.Delayed', 'Statistics.Flights.Diverted',
       'Statistics.Flights.On Time', 'Statistics.Flights.Total',
       'Statistics.Minutes Delayed.Carrier',
       'Statistics.Minutes Delayed.Late Aircraft',
       'Statistics.Minutes Delayed.National Aviation System',
       'Statistics.Minutes Delayed.Security',
       'Statistics.Minutes Delayed.Total',
       'Statistics.Minutes Delayed.Weather', 'Time.Label', 'Time.Month',
       'Time.Month Name', 'Time.Year',
       'Statistics.Flights.Proportion Delayed'],
      dtype='object')

In [ ]:
app = JupyterDash(__name__)
app.layout = html.Div([
    html.Div([
        html.H1("Airport Dashboard"),
        html.Label([
            "Airport Name",
            dcc.Dropdown(
                id='airport-name-dropdown', clearable=False,
                value='San Francisco International', options=[
                    {'label': name, 'value': name}
                    for name in df['Airport.Name'].sort_values().unique()
                ])
            ]),
        dcc.Graph(id='graph'),
    ])
])
# Define callbacks to update graphs
@app.callback(
    Output('graph', 'figure'),
    [Input("airport-name-dropdown", "value")]
)
def update_figure(val):
    try:
        #print(val)
        d = df[df['Airport.Name'] == val]
        fig = px.scatter_geo(geo_df, lon=geo_df.geometry.x, lat=geo_df.geometry.y, color='Chance of Delay', hover_data=['Airport.Name'])
        fig.update_traces(marker=dict(size=12))
        return fig
    except:
        return None

In [ ]:
px.scatter_geo(geo_df, lon=geo_df.geometry.x, lat=geo_df.geometry.y, color='Chance of Delay', hover_data=['Airport.Name'])

AttributeError: ignored

In [ ]:
app.run_server(mode='inline')

<IPython.core.display.Javascript object>

## Vanilla Dash attempt (not working in .ipynb)

In [ ]:
!pip install dash
import dash
from dash import dcc
from dash import html
import pandas as po

data = po.read_csv("data/Cleaned_airports.csv", parse_dates=True)


app = dash.Dash(__name__)

In [ ]:
app.run_server(debug=True)

SystemExit: 1

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3445: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.

